1. As mentioned earlier, we extracted user-movie relationships only from the
movie rating data where most ratings are unknown. Can you also utilize
data from the files movies.dat and users.dat

In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from pprint import pprint
from IPython.display import display

In [ ]:
در این قسمت کتابخانه های مورد نیاز اضافه شده است

#### Movie Recommendation Engine (Naive Bayes)

In [2]:
df_rating=pd.read_csv("ml-1m/ratings.dat", sep="::",names=["UserID","MovieID","Rating","Timestamp"])
df_user=pd.read_csv("ml-1m/users.dat", sep="::",names=["UserID","Gender","Age","Occupation","Zip-code"])
df_movie=pd.read_csv("ml-1m/movies.dat", sep="::",names=["MovieID","Title","Genres"])

C:\Users\sepah\AppData\Local\Temp\ipykernel_11204\4183090225.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_rating=pd.read_csv("ml-1m/ratings.dat", sep="::",names=["UserID","MovieID","Rating","Timestamp"])
C:\Users\sepah\AppData\Local\Temp\ipykernel_11204\4183090225.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_user=pd.read_csv("ml-1m/users.dat", sep="::",names=["UserID","Gender","Age","Occupation","Zip-code"])
C:\Users\sepah\AppData\Local\Temp\ipykernel_11204\4183090225.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (se

In [ ]:
در این قسمت اطلاعات از سه جدول مختلف خوانده شده و در سه دیتافریم مختلف ذخیره شده است

In [3]:
display(df_user.head(2))
display(f"{df_user.shape,=}")

display(df_movie.head(2))
display(f"{df_movie.shape,=}")

display(df_rating.head(2))
print(f"{df_rating.shape,=}")

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072


'df_user.shape,=((6040, 5),)'

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


'df_movie.shape,=((3883, 3),)'

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109


df_rating.shape,=((1000209, 4),)


In [ ]:
محتویات سه دیتا فریم
df_user, df_movie, df_rating
و تعداد رکوردها و فیلدها نمایش داده شده است

In [4]:
print(f"{df_user['UserID'].nunique(),=}")
print(f"{df_user['Age'].nunique(),=}")
print(f"{df_user['Occupation'].nunique(),=}")
print(f"{df_movie['MovieID'].nunique(),=}")
print()
print(f"{df_rating['UserID'].nunique(),=}")
print(f"{df_rating['MovieID'].nunique(),=}")

df_user['UserID'].nunique(),=(6040,)
df_user['Age'].nunique(),=(7,)
df_user['Occupation'].nunique(),=(21,)
df_movie['MovieID'].nunique(),=(3883,)

df_rating['UserID'].nunique(),=(6040,)
df_rating['MovieID'].nunique(),=(3706,)


In [ ]:
تعداد کاربران، سن، شغل و فیلم ها به صورت منحصر به فرد مشخص گردیده است

In [5]:
df_movie["Genres"]=df_movie["Genres"].str.split("|")
Genres=set(df_movie["Genres"].explode())
print(f"{len(Genres),=}")
print(f"{Genres,=}")

len(Genres),=(18,)
Genres,=({'Musical', 'Action', 'War', 'Horror', 'Romance', 'Fantasy', 'Sci-Fi', 'Adventure', "Children's", 'Drama', 'Animation', 'Documentary', 'Mystery', 'Film-Noir', 'Crime', 'Thriller', 'Western', 'Comedy'},)


In [ ]:
در این قسمت ژانر فیلم ها مشخص شده است

In [12]:
#User Preprocess:
Occupation=pd.get_dummies(df_user["Occupation"], prefix="O").astype(int)
Age=pd.get_dummies(df_user["Age"], prefix="Age").astype(int)
df_user["Gender"]=df_user["Gender"].replace({"F":0, "M":1}).astype(int)

df_user_p = pd.concat([df_user, Occupation], axis=1)
df_user_p = pd.concat([df_user_p, Age], axis=1)
df_user_p=df_user_p.drop(columns=["Occupation","Zip-code", "Age"])
df_user_p.head(2)

,UserID,Gender,O_0,O_1,O_2,O_3,O_4,O_5,O_6,O_7,...,O_18,O_19,O_20,Age_1,Age_18,Age_25,Age_35,Age_45,Age_50,Age_56
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
در این قسمت User Preprocess 
از قبیل onehot
کردن شغل و سن
و تبدیل جنسیت انجام شده و در نهایت دیتافریم
df_user_p
ایجاد گردیده است

In [6]:
display(df_movie.head(2))
print(f"{df_movie.shape,=}")
print(f"{len(Genres),=}")

,MovieID,Title,Genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"


df_movie.shape,=((3883, 3),)
len(Genres),=(18,)


In [ ]:
دیتا فریم df_movie نمایش داده شده

In [7]:
# Movie Preprocess:
from sklearn.preprocessing import MultiLabelBinarizer
Genres_Binarizer=MultiLabelBinarizer().fit_transform(df_movie["Genres"])
col=[f"G_{i}" for i in range(len(Genres))]
Genres_Binarizer=pd.DataFrame(Genres_Binarizer, columns=col).astype(int)
print(Genres_Binarizer.shape)
display(Genres_Binarizer.head(2))

(3883, 18)


,G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
پیش پردازش های Movie
انجام شده ستون ژانر را از طریق
MultiLabelBinarizer
پیش پردازش کرده ایم
و درنهایت آن را نمایش داده ایم

In [10]:
df_movie_p = pd.concat([df_movie, Genres_Binarizer], axis=1)
df_movie_p = df_movie_p.drop(columns=["Genres", "Title"])
df_movie_p.head(2)

,MovieID,G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
پس از پیش پردازش انجام شده بر روی df_movie
ستون های عنوان و ژانر را حذف می کنیم
ستون عنوان نیاز به پردازش متن داشته که خارج از حیطه این فصل کتاب می باشد
در نهایت df_movie_p را می سازیم

In [13]:
display(df_rating.head(2))
display(df_movie_p.head(2))
display(df_user_p.head(2))

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109


,MovieID,G_0,G_1,G_2,G_3,G_4,G_5,G_6,G_7,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


,UserID,Gender,O_0,O_1,O_2,O_3,O_4,O_5,O_6,O_7,...,O_18,O_19,O_20,Age_1,Age_18,Age_25,Age_35,Age_45,Age_50,Age_56
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
دیتا فریم ها نمایش داده شده است

In [14]:
df = pd.merge(df_rating, df_user_p, on="UserID", how="left")
df = pd.merge(df, df_movie_p, on="MovieID", how="left")
df=df.drop(columns=["Timestamp"])
print(df.shape)
df.head(2)

(1000209, 50)


,UserID,MovieID,Rating,Gender,O_0,O_1,O_2,O_3,O_4,O_5,...,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17
0,1,1193,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,661,3,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
اطلاعات در دیتا فریم
df_rating
تجمیع شده و ستون 
timestamp را
حذف می کنیم

In [15]:
print(df["Rating"].value_counts())
print()
print(df["Rating"].value_counts(normalize=True))

4    348971
3    261197
5    226310
2    107557
1     56174
Name: Rating, dtype: int64

4    0.348898
3    0.261142
5    0.226263
2    0.107535
1    0.056162
Name: Rating, dtype: float64


In [ ]:
توزیع پراکندگی امتیازات را مشاهده کرده

In [16]:
th=1000
movie_id=df["MovieID"].value_counts().sort_values(ascending=False)
movie_id_index=movie_id[movie_id>th].index
df=df[df["MovieID"].isin(movie_id_index)]
print(f"{df.shape,=}")

df.shape,=((305373, 50),)


In [ ]:
با توجه به کتاب و اینکه تعداد نظرات برای برخی از فیلم ها کم می باشد ما فقط فیلم هایی با حداقل تعداد 1000 نظر را در نظر می گیریم
تعداد رکوردها به 305 هزار عدد می رسد

In [17]:
X=df.drop(columns=["Rating"])
y=df["Rating"].astype(int)

display(X.head(2))
display(y.head(2))

,UserID,MovieID,Gender,O_0,O_1,O_2,O_3,O_4,O_5,O_6,...,G_8,G_9,G_10,G_11,G_12,G_13,G_14,G_15,G_16,G_17
0,1,1193,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3408,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    5
3    4
Name: Rating, dtype: int32

In [ ]:
x,y 
را مشخص کرده

In [18]:
recommend = 3
y[y <= recommend] = 0
y[y > recommend] = 1
n_pos = (y == 1).sum()
n_neg = (y == 0).sum()
print(f'{n_pos:,} positive samples and {n_neg:,} negative... samples.')

216,634 positive samples and 88,739 negative... samples.


In [ ]:
با توجه به کتاب امتیازات 4و5 را جزء گروه 1
و امتیازات زیر 3 را جزء گروه 0 
در نظر می گیریم

In [19]:
from sklearn.model_selection import train_test_split
test_size=0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, stratify=y)  #

X_train, y_train = X_train.to_numpy(), y_train.to_numpy()
X_test,  y_test  = X_test.to_numpy(), y_test.to_numpy()

print(f"{X_train.shape,=}\t{X_test.shape,=}")
print(f"{y_train.shape,=}\t{y_test.shape,=}")

X_train.shape,=((213761, 49),)	X_test.shape,=((91612, 49),)
y_train.shape,=((213761,),)	y_test.shape,=((91612,),)


In [ ]:
با نسبت 70 به 30 داده های آموزش و آزمایش را 
جدا می کنیم

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
با توجه به نرمال نبودن داده ها از کتابخانه
StandardScaler
برای نرمالسازی داده ها استفاده می کنیم

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred_prob = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [ ]:
در اینجا مدل مربوطه را برای طبقه بند مشخص میکنیم ما از دو مدل
LogisticRegression
GaussianNB
استفاده کرده ایم

In [22]:
i=5
print(y_test[i])
print(y_pred[i])
print(y_pred_prob[i])

1
1
[0.25157992 0.74842008]


In [ ]:
احتمالات پیش بینی شده برای داده اندیس 5
را به همراه کلاس پیش بینی شده و کلاس واقعی را نمایش می دهیم

In [23]:
from sklearn import metrics

print(f"Accuracy Score: {round(metrics.accuracy_score(y_test, y_pred), 2)}")
print(f"F1 Score: {round(metrics.f1_score(y_test, y_pred),2)}")
print(f"ROC AUC Score: {round(metrics.roc_auc_score(y_test, y_pred),2)}")

Accuracy Score: 0.71
F1 Score: 0.83
ROC AUC Score: 0.5


In [ ]:
معیار ارزیابی ها را نمایش می دهیم